# <center>文档说明</center>

本笔记本文档仅用于测试基于sumo环境的仿真评价结果获取与可视化分析，不作为项目最终成果的一部分。

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import xml.dom.minidom
from xml.dom.minidom import parse

In [77]:
file = "./output/output-tripinfos.xml"
tree = parse(file)
collection = tree.documentElement

In [78]:
vehinfos = collection.getElementsByTagName("tripinfo")

In [79]:
# 结果表格化输出
id_, dir_, delay_, wtime_ = [],[],[],[]
for veh in vehinfos:
    vehid = veh.getAttribute("id")
    id_.append(vehid)
    dir_.append(vehid[0:2])
    delay_.append(float(veh.getAttribute("timeLoss")))
    wtime_.append(float(veh.getAttribute("waitingTime")))
result = pd.DataFrame({"id":id_, "direction": dir_, "delay": delay_, "waitingTime": wtime_})

In [80]:
delay = result.groupby('direction')['delay'].agg(['mean','count'])
wt = result.groupby('direction')['waitingTime'].agg(['mean','count'])
round((delay['mean'] * delay['count']).sum() / delay['count'].sum(), 2)

69.2

In [13]:
wt

,mean,count
direction,,
EN,0.496654,269
ES,60.014000,300
EW,51.339338,544
NE,74.017647,340
NS,34.185531,622
NW,32.291824,159
SE,48.479070,86
SN,50.421370,365
SW,61.252381,42


In [112]:
# 动态交互获取延误指标
import os, sys
import numpy as np
# Check sumo env
if 'SUMO_HOME' in os.environ:
     tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
     sys.path.append(tools)
else:
     sys.exit("please declare environment variable 'SUMO_HOME'")
import traci
seed = 20200720
sumoCmd = ['sumo', '-c', './ITLS.sumo.cfg', '--start', '--seed', str(seed), '--quit-on-end']
traci.start(sumoCmd, label='training')
scenario = traci.getConnection('training')

ent_lanes = ['CAEE', 'CAWE', 'JSNE', 'JSSE']
ext_lanes = ['CAEX', 'CAWX', 'JSNX', 'JSSX']
lanearea = [
        'W.01', 'W.02','W.03', 'W.04', 'W.05',
        'E.01', 'E.02','E.03', 'E.04', 'E.05', 'E.06',
        'S.01', 'S.02', 'S.03',
        'N.01', 'N.02', 'N.03'
    ]

while traci.simulation.getTime() <= 60:
    traci.simulationStep()

time_series, values = [], []
while traci.simulation.getTime() <= 3660:
    delay = dict.fromkeys(ent_lanes, 0)
    ql = dict.fromkeys(['W','E','S','N'], 0)
    vol = 0
    time = traci.simulation.getTime()
    for area in lanearea:
        _dir = area[0]
        ql[_dir] += traci.lanearea.getJamLengthMeters(area)
        #print("*******",time,"*******")
        #print(sum(delay.values())/len(delay))
    for edge in ent_lanes:
        d = 0
        for indx in range(1, traci.edge.getLaneNumber(edge)):
            lane = edge + '_' + str(indx)
            d += 1 - (traci.lane.getLastStepMeanSpeed(lane)/traci.lane.getMaxSpeed(lane))
        delay[edge] += d * 0.2 / (traci.edge.getLaneNumber(edge) - 1)
    for ext in ext_lanes:
        vol += traci.edge.getLastStepVehicleNumber(ext)
    
    print(np.sum(list(delay.values())), np.std(list(delay.values())))
    print(np.sum(list(ql.values())), np.std(list(ql.values())))
    print(vol)
    traci.simulationStep()

traci.close()

0.43833553910560363 0.04089528916065923
308.30056006648 60.415775691251305
2
0.44021436468678266 0.040562862213450554
308.30056006648 60.415775691251305
2
0.44174453224355137 0.04026870016294227
308.30056006648 60.415775691251305
2
0.4432855578572814 0.039969979315782306
308.30056006648 60.415775691251305
2
0.4440767283401459 0.03992681549701489
308.30056006648 60.415775691251305
1
0.4358329832770562 0.03788385408439576
308.30056006648 60.415775691251305
1
0.4368474396995528 0.0378191719317612
308.30056006648 60.415775691251305
1
0.43793645046413854 0.03773278289059696
308.30056006648 60.415775691251305
1
0.43851339126729016 0.03769463105785805
308.30056006648 60.415775691251305
1
0.42982229187970694 0.04096255144924416
308.30056006648 60.415775691251305
1
0.4272368036721369 0.041227582846054156
308.30056006648 60.415775691251305
0
0.42824894270469666 0.04108936018191942
308.30056006648 60.415775691251305
0
0.43322962673244136 0.03957338183237665
308.30056006648 60.415775691251305
0
0.

0.5499567131478615 0.02957896495460146
308.03268934975966 33.52241757346739
5
0.5515967483822866 0.029903751551412984
303.00177015672983 34.09529333988721
5
0.5560470179192667 0.030020419678505106
302.9769622262247 34.092429545850855
5
0.5506014358691412 0.03103636446950732
302.9570656601816 34.09013282862818
5
0.5520391743777948 0.03143847000945523
302.94111507159755 34.08829163718823
4
0.5534848970000475 0.03184991458030758
309.83750651123626 31.83114283322421
5
0.551157326299409 0.030867066871833566
309.759056578263 31.85169093016312
5
0.5526286278159928 0.0312893972086488
309.69584979288095 31.868275265071002
5
0.5500738634537334 0.03297878248165305
309.6449995447184 31.881635576754075
6
0.5517561591234832 0.03337025563758586
309.60413839116416 31.892382679690968
6
0.5448867608421778 0.03317339960205486
309.5713347791037 31.901017539164307
6
0.5464567986763871 0.03351298074467287
323.01884032631006 33.246579590359474
5
0.5475370848775302 0.03381751301430303
329.8299436739967 32.173

0.5295341991276667 0.04864720837028672
376.97707531510474 49.85112319849431
3
0.5302658943573415 0.04918893784116465
377.0727978579283 49.815266550393275
3
0.5322379491270359 0.049311770170640604
384.1172065856531 52.09756924863332
3
0.5324948020502288 0.049956939742053515
384.2009902503597 52.02039674241409
3
0.5362936718227916 0.04926133235584065
383.5928510380912 53.93819434771261
3
0.5366122267645033 0.049875260203526756
383.5573685887996 53.89669306247757
3
0.5369772632861026 0.050464424610742575
388.5640459384703 54.389434389681114
3
0.5374035813135534 0.051025270957322795
403.59010688795354 56.422423690191444
4
0.5373056504766403 0.0513858491552833
403.4965337620388 56.34059154176921
4
0.5373286977673343 0.051725637920894185
403.33912862097924 56.238659246408645
4
0.5374881621571233 0.05204527738264934
382.90954894708534 57.18023980188373
4
0.5393432185484638 0.05185283746112046
389.90820361388705 59.36343415594348
5
0.5414517366426633 0.05162070083507112
389.8496000298686 59.30

0.5193843588734436 0.05034140830560733
454.53392659243326 66.94576969790975
4
0.5210810748674304 0.04982771587923746
454.66854138591776 66.9685448161855
5
0.52293169112772 0.0492846659743544
454.8757074774081 67.00139055987509
5
0.5249321659694154 0.048720066687335584
455.1617448051846 67.04547683111323
5
0.5267473036755262 0.048235477122602614
455.53258760710366 67.10195263020259
4
0.537080453136187 0.04935677555215204
448.49858346860174 66.123088933279
4
0.5391234453630493 0.04911369183117896
442.24075691756144 65.73191516164519
4
0.5410392765292386 0.04893578037540502
433.06460032181735 64.57245342474765
4
0.5279357597065303 0.05317718021795306
440.3056924115312 65.13357261365611
4
0.5296238940585569 0.05286840670221913
440.5524675069992 65.16200605269216
4
0.5352831421315863 0.05184234560706275
440.875265936318 65.19876728909867
4
0.537306262278497 0.05151344741167394
441.2754799826301 65.24425692555317
4
0.5106631934116503 0.06034638515704539
441.7545033617669 65.29891885436386
4


0.4651452498188153 0.06548502982633797
399.60914396490926 91.84369576835228
5
0.46541727385228493 0.06559468317305521
399.42529082235535 91.83758988077383
5
0.46549470311521113 0.06557996168094804
399.27700395100953 91.83272126916692
5
0.4656184988817629 0.06556446956972885
399.1575976143363 91.82883551417551
5
0.465902913624389 0.0655243253085198
399.06157189045496 91.825731658302
5
0.46626383705895547 0.06547941486160773
398.9844284163603 91.82325066565556
5
0.466741321327801 0.06542235670108634
398.9225049475415 91.82126642711756
5
0.46719526165264863 0.06538595718505623
398.87283084451514 91.81967875702777
4
0.4677057049691391 0.06535380067822655
398.83300341531765 91.81840793288266
4
0.46825944871930003 0.06532964635560695
398.8010842135685 91.81739032120862
4
0.49162464584463045 0.059772467850130816
398.7755147463971 91.81657627962174
4
0.4884130016017008 0.05861038401578399
398.7550343434049 91.81592413214793
4
0.48906495121283844 0.05865327324922431
398.7386334605987 91.8154016

...

0.4496775617220984 0.05091331114638274
259.43019354804426 76.47623462558067
8
0.45167969196516056 0.05109348869817806
259.42988923588644 76.47610498037928
7
0.45297109934065927 0.05119442093831976
264.4296458152328 75.71243453109537
8
0.4538520335349936 0.05123850392406868
264.42945110321466 75.712351545604
6
0.45407822884737425 0.05120004337974637
264.42929535399804 75.71228516590651
6
0.46604648234124707 0.04604592258935407
271.41787155918036 78.69260610914512
6
0.4904976729940762 0.034714281720033505
271.3343989002875 78.65698803773452
6
0.4916525190791635 0.0344839882201394
271.26693597449287 78.62820172586939
7
0.4936708498184024 0.03434275125956538
271.21252401418093 78.60498446702798
8
0.4893900178721603 0.03350074515411698
271.1687104262953 78.58628961921218
8
0.5153426891341646 0.038036314497577224
271.13347708808357 78.57125597994127
8
0.5175625506462278 0.03824153882944292
224.09331861602715 58.60261241460838
8
0.5199887510754848 0.03845186749302555
224.0932925154925 58.6026

0.4762987765607378 0.027854907119663718
328.24898639623336 77.63952114117642
0
0.47635729276758815 0.028165084492106265
328.451367036231 77.72970126732746
0
0.476281341236664 0.028468598000442193
328.6870313455353 77.83332876116867
0
0.4760690323085499 0.028763650441255804
321.21017059465885 74.64680177641196
0
0.4761550559642571 0.028916353995774382
321.2571551443975 74.66877708623227
0
0.4764625329219261 0.02895192803941593
321.29480676289836 74.68638517075516
0
0.4762426186872641 0.029102639392343376
321.3244147309091 74.70025316053847
0
0.4763974023361531 0.029097309723038332
321.34716430769026 74.71094842124643
0
0.47023624672331815 0.027060217392869672
321.36401594889776 74.718930124163
0
0.4722799061378385 0.02735970594580393
328.31280614034296 74.09831370706887
0
0.4742239364924325 0.02765830130855543
333.2462048150667 73.53216200508116
0
0.47403370714844834 0.02760058051309523
333.1894178392463 73.53882917849138
0
0.4737961614674988 0.027545442044828918
353.6647321841614 69.83

0.5177940640278442 0.04098071058733335
386.8523811807684 68.35543370708214
1
0.5174438345559753 0.041273246602015916
387.1414922164896 68.51737492167415
1
0.516474115593765 0.04157329646999133
377.96841152780894 64.63224653299486
1
0.515316616954967 0.041879433195230256
378.2604501312246 64.78156357875955
1
0.5142017992689407 0.04219314191837225
385.5900334477751 64.34959478618485
1
0.5186183980182251 0.041251713693998306
392.9311329998378 64.08899455294198
1
0.5178366596770272 0.041530664584815656
385.7284653575686 61.14497123866265
1
0.5171096671259214 0.04183510357031067
386.06342346331525 61.349160887919744
1
0.5164424315044616 0.04216379739138972
378.69657556020593 58.31628538812169
1
0.5158291907438171 0.04251270754143705
379.02883883201196 58.49598870194348
1
0.5152581079227148 0.042875069317575516
379.4500041690694 58.704895994024696
1
0.5038046184342265 0.04094693950968982
372.33856127731696 55.76281562339904
1
0.5034094007134051 0.041317549224834806
386.6620175224982 61.77410

0.5402909354339506 0.058159660843892215
255.5393163469295 40.06365695045938
9
0.5408517782196605 0.0584724647743578
255.45493146494746 40.06249127166494
8
0.5415107971942079 0.05879165297932645
255.38705682570523 40.061578925365595
8
0.5207047917538875 0.054895729147517426
255.33252478170616 40.060862218083436
7
0.5213496374281343 0.05495849535903428
255.28875248521902 40.060297412561255
7
0.5218311150454596 0.05504702719898679
255.25364237348802 40.05985111899042
7
0.5372648330585226 0.05032328328412234
255.22549642810517 40.05949767734383
7
0.5390829040841066 0.04995512813691735
255.20294354153478 40.059217247385604
7
0.5398592991904577 0.0498814836586155
255.1848821747811 40.05899535504254
7
0.540718045084917 0.04976031900309208
255.17041922428567 40.058818811295325
7
0.5416488454293767 0.049597333369561805
255.15884038693773 40.058678203362106
8
0.5434407571146465 0.0491604409539872
255.14957219513977 40.058566122682905
8
0.5445194939001596 0.04892264295478979
255.14215460070162 40

0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
5
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
4
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
3
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
3
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
4
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
4
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
4
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
3
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
3
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
3
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
3
0.4166666666666667 0.08196798155377501
215.7907047671909 55.217007374534234
3
0.4166666666666667 0.08196798155377501
215.7907047671909 55.2170

In [111]:
traci.close()

In [109]:
list(delay.values()

AttributeError: 'dict_values' object has no attribute 'tolist'

In [53]:
import numpy as np

A = np.mat(
    [
        [1, -11.64, (-11.64)**2, -11.64**3],
        [1, 3.16, 3.16**2, 3.16**3],
        [1, 17.54, 17.54**2, 17.54**3],
        [1, 31.50, 31.50**2, 31.50**3]
    ]
)
b = np.mat([75.15, 67.69, 67.08, 73.34]).T
c = np.matmul(np.linalg.inv(A), b)

x = np.linspace(-14.05, 48.00, 5)
y = np.matmul(np.mat([(x+2.41)**0, x-2.41, (x+2.41)**2, (x+2.41)**3]).T, c).T
x, np.array(y)[0] -4.08

(array([-14.05  ,   1.4625,  16.975 ,  32.4875,  48.    ]),
 array([72.88399547, 65.23446125, 65.24159514, 73.68396402, 91.34013476]))

In [118]:
with parse("./ITLS.net.xml") as tree:
    collection = tree.documentElement
    edges = collection.getElementsByTagName("edge")
    for edge in edges:
        print(edge.getAttribute("id"))

:EFlared_0
:EFlared_7
:ITLS_0
:ITLS_1
:ITLS_2
:ITLS_3
:ITLS_4
:ITLS_6
:ITLS_34
:ITLS_7
:ITLS_8
:ITLS_9
:ITLS_10
:ITLS_11
:ITLS_15
:ITLS_16
:ITLS_35
:ITLS_17
:ITLS_18
:ITLS_19
:ITLS_20
:ITLS_21
:ITLS_23
:ITLS_36
:ITLS_24
:ITLS_25
:ITLS_26
:ITLS_27
:ITLS_28
:ITLS_32
:ITLS_33
:ITLS_37
:NFlared_0
:NFlared_4
:SFlared_0
:SFlared_3
:WFlared_0
:WFlared_5
CAED
CAEE
CAEU
CAEX
CAWD
CAWE
CAWU
CAWX
JSND
JSNE
JSNU
JSNX
JSSD
JSSE
JSSU
JSSX


In [117]:
file = "./ITLS.add.xml"
tree = parse(file)
collection = tree.documentElement
laneareas = collection.getElementsByTagName("laneAreaDetector")
laneareas.getAttribute("id")
#for lanearea in laneareas:
#    print(lanearea.getAttribute("id"))

AttributeError: 'NodeList' object has no attribute 'getAttribute'